##Decision Tree

Tree based learning algorithms are considered to be one of the best and mostly used supervised learning methods. Tree based methods empower predictive models with high accuracy, stability and ease of interpretation. Unlike linear models, they map non-linear relationships quite well. They are adaptable at solving any kind of problem at hand (classification or regression).

Decision tree is a type of supervised learning algorithm (having a pre-defined target variable) that is mostly used in classification problems. It works for both categorical and continuous input and output variables. In this technique, we split the population or sample into two or more homogeneous sets (or sub-populations) based on most significant splitter / differentiator in input variables.

How does a tree decide where to split?
The decision of making strategic splits heavily affects a tree’s accuracy. The decision criteria is different for classification and regression trees.

Decision trees use multiple algorithms to decide to split a node in two or more sub-nodes. The creation of sub-nodes increases the homogeneity of resultant sub-nodes. In other words, we can say that purity of the node increases with respect to the target variable. Decision tree splits the nodes on all available variables and then selects the split which results in most homogeneous sub-nodes. Gini Index is one of the methods for this.

In [59]:
import pandas as pd
import numpy as np
import random

In [60]:
df=pd.read_csv('iris.csv')

#replacing all missing data with NaN value
df.replace('?',np.nan,inplace=True)

#deleting the column with id, 1 in the argument indicates 'column', so this will delete the 'column' containing 'id'
#df.drop(['id'],1,inplace=True)

#delete all the rows that have NaN in them
dk=df.dropna()
full_data=dk.astype(float).values.tolist()
headers = df.dtypes.index
header=headers.tolist()

In [61]:
header

['x1', 'x2', 'label']

In [62]:
#shuffle the data, not necessary but if done the results are more accurate
random.shuffle(full_data)

test_size1=0.1
train_data1=full_data[:-int(test_size1*len(full_data))]
test_data1=full_data[-int(test_size1*len(full_data)):]

test_size2=0.5
train_data2=full_data[:-int(test_size2*len(full_data))]
test_data2=full_data[-int(test_size2*len(full_data)):]

test_size3=0.3
train_data3=full_data[:-int(test_size3*len(full_data))]
test_data3=full_data[-int(test_size3*len(full_data)):]

In [63]:
class Question:
    def __init__(self,column,value):
        '''function to take the value and the index of the element in the list
        eg: suppose the list is ['Green',3,'Apple'], now if column=1 and value='Green' '''
        self.column=column
        self.value=value
    
    def match(self,example):
        '''function to check the element at the column of the passed list and compare it to the value passed to the question
        if column=1 and value='Green' and the above list is passed then the function will return True '''
        val=example[self.column]
        return val==self.value
    
    def __repr__(self):
        return "Is %s == %s ?" %(header[self.column],str(self.value))

In [64]:
#DTF is Decision tree functions
class DTF:            
    def unique_values(self,data,col):
        '''function to take dataset as agrument and returns a set of the elements in the dataset'''
        return (set(row[col] for row in data))
    
    
    def class_counts(self,data):
        '''fucntion to count the number of times the element(output) is present in the set'''
        self.counts={}
        for row in data:
            self.label=row[-1]
            
            #if item isn't present in the dictionary then initialize its count to 0
            
            if self.label not in self.counts:
                self.counts[self.label]=0
                
            #increment the count of the element that is found
            
            self.counts[self.label]+=1
        return self.counts
    
    def is_numeric(self,value):
        '''function returns True if the value is int or float'''
        return(isinstance(self.value,int) or isinstance(self.value,float))
    
    def partition(self,rows, question):
        '''function to generate true_list and false_list by appending rows that satisfy the question'''
        self.true_rows,self.false_rows=[],[]
        for row in rows:
            if question.match(row):
                self.true_rows.append(row)
            else:
                self.false_rows.append(row)
        return self.true_rows,self.false_rows
    
    def gini(self,rows):
        self.counts=self.class_counts(rows)
        self.impurity=1
        for output in self.counts:
            self.probability_output=self.counts[output]/float(len(rows))
            self.impurity-=self.probability_output**2
        return self.impurity
    
    def info_gain(self,left,right,current_uncertainity):
        self.p=len(left)/float(len(left)+len(right))
        return (current_uncertainity-self.p*self.gini(left)-(1-self.p)*self.gini(right))
        
    def find_best_split(self,rows):
        self.best_gain=0
        self.best_question=None
        
        self.current_uncertainity=self.gini(rows)
        self.n_features=len(rows[0])-1
        
        for col in range(self.n_features):
            self.values=self.unique_values(rows,col)
            
            for val in self.values:
                self.question=Question(col,val)
                self.true_rows,self.false_rows=self.partition(rows,self.question)
                if len(self.true_rows)==0 or len(self.false_rows)==0:
                    continue
                self.gain=self.info_gain(self.true_rows,self.false_rows,self.current_uncertainity)
                if self.gain>=self.best_gain:
                    self.best_gain=self.gain
                    self.best_question=self.question
                    
            return self.best_gain,self.best_question

In [65]:
class Leaf:
    def __init__(self,rows):
        self.DTF=DTF()
        self.prediction=self.DTF.class_counts(rows)
        
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch

In [66]:
class Decision_Tree(DTF):
    def __init__(self):
        self.DTF=DTF() 
                
    def build_tree(self,training_data):
        self.gain,self.question=self.DTF.find_best_split(training_data)
        if self.gain==0:
            return Leaf(training_data)
        
        self.true_rows, self.false_rows=self.DTF.partition(training_data,self.question)
        self.DecisionNode=Decision_Node(self.question,self.build_tree(self.true_rows),self.build_tree(self.false_rows))
        return self.DecisionNode
    
    def print_tree(self,node,spacing=''):
        if isinstance(node,Leaf):
            print (spacing,'Predict',node.prediction)
            return
        print (spacing,str(node.question))
        print (spacing,'--> True')
        self.print_tree(node.true_branch,spacing+'  ')
        print (spacing,'--> False')
        self.print_tree(node.false_branch,spacing+'  ')
        
    def classify(self,row,node):
        if isinstance(node,Leaf):
            return node.prediction
        if node.question.match(row):
            return self.classify(row,node.true_branch)
        else:
            return self.classify(row,node.false_branch)
        
    def print_leaf(self,counts):
        self.total=sum(counts.values())*1.0
        self.probs={}
        for lbl in counts.keys():
            self.probs[lbl]=str(int(counts[lbl]/self.total*100))+'%'
        return self.probs
    
    def predict(self,testing_data,node):
        for row in testing_data:
            print (row[-1], self.print_leaf(self.classify(row,node)))
            
    def accuracy(self,testing_data,node):
        self.correct=0
        self.all=0
        for row in testing_data:
            self.result=self.print_leaf(self.classify(row,node))
            if row[-1]==max(self.result, key=self.result.get):
                self.correct+=1
        return self.correct/float(len(testing_data))*100

In [67]:
dt1=Decision_Tree()
dt2=Decision_Tree()
dt3=Decision_Tree()

In [68]:
my_tree1=dt1.build_tree(train_data1)
my_tree2=dt2.build_tree(train_data2)
my_tree3=dt3.build_tree(train_data3)

In [76]:
dt1.print_tree(my_tree1)

 Is x1 == 1.5 ?
 --> True
   Predict {0.0: 13}
 --> False
   Is x1 == 1.4 ?
   --> True
     Predict {0.0: 12}
   --> False
     Is x1 == 1.3 ?
     --> True
       Predict {0.0: 7}
     --> False
       Is x1 == 1.6 ?
       --> True
         Predict {0.0: 7}
       --> False
         Is x1 == 1.7 ?
         --> True
           Predict {0.0: 4}
         --> False
           Is x1 == 1.2 ?
           --> True
             Predict {0.0: 2}
           --> False
             Is x1 == 1.9 ?
             --> True
               Predict {0.0: 2}
             --> False
               Is x1 == 1.1 ?
               --> True
                 Predict {0.0: 1}
               --> False
                 Is x1 == 1.0 ?
                 --> True
                   Predict {0.0: 1}
                 --> False
                   Predict {1.0: 41}


In [77]:
dt1.print_tree(my_tree2)

 Is x1 == 1.4 ?
 --> True
   Predict {0.0: 7}
 --> False
   Is x1 == 1.5 ?
   --> True
     Predict {0.0: 7}
   --> False
     Is x1 == 1.6 ?
     --> True
       Predict {0.0: 5}
     --> False
       Is x1 == 1.7 ?
       --> True
         Predict {0.0: 3}
       --> False
         Is x1 == 1.3 ?
         --> True
           Predict {0.0: 3}
         --> False
           Is x1 == 1.1 ?
           --> True
             Predict {0.0: 1}
           --> False
             Predict {1.0: 24}


In [78]:
print (100-test_size1*100,'% training data','Accuracy',dt1.accuracy(test_data1,my_tree1))
print (100-test_size2*100,'% training data','Accuracy',dt2.accuracy(test_data2,my_tree2))
print (100-test_size3*100,'% training data','Accuracy',dt3.accuracy(test_data3,my_tree3))

90.0 % training data Accuracy 100.0
50.0 % training data Accuracy 90.0
70.0 % training data Accuracy 93.33333333333333
